![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

This notebook demonstrates how to create a video + audio pipeline using the GStreamer multimedia framework with two different source options:
* The first option uses an encoded video file as source. This capture pipeline is based on ``multifilesrc`` and ``decodebin`` which demuxes the video and audio streams of the video file.
* The second option captures video/audio from an incoming HDMI stream. This capture pipeline is based on ``mediasrcbin`` for video and ``alsasrc`` for audio.

On the sink side, ``kmssink`` is used for video and ``alsasink`` for audio.

In this notebook, you will:
1. Create a GStreamer pipeline that includes a video and audio path using the ``parse_launch()`` API
2. Create a GStreamer pipeline graph and view it inside this notebook.

# 2. Imports and Initialization

Import all python modules required for this notebook. 

In [ ]:
from IPython.display import Image, display, clear_output
import glob
import subprocess
import pydot
import sys
import gi
import re
gi.require_version('Gst', '1.0')
from gi.repository import GObject, GLib, Gst

This is the Base TRD notebook 4 (nb4).

In [ ]:
nb = "nb8"

Create a directory for saving the pipeline graph as dot file. Set the GStreamer debug dot directory environment variable to point to that directory.

In [ ]:
dotdir = "/home/root/gst-dot/" + nb
!mkdir -p $dotdir
%env GST_DEBUG_DUMP_DOT_DIR = $dotdir

Initialize the GStreamer library. Enable debug by setting the debug string, set default to level 1 for all categories.

In [ ]:
Gst.init(None)
Gst.debug_set_threshold_from_string('*:1', True)

# 3. Create String Representation of GStreamer Pipeline

Describe the ``kmssink`` and ``alsasink`` elements and their properties as string representation.

In [ ]:
plane_id = 38
vsink = "kmssink plane-id=" + str(plane_id)
asink = "alsasink device=hw:0,0"

The ``get_media_by_device`` function returns the matching media node for a given video capture source.

In [ ]:
def get_media_dev_by_name(src):
    sources = {
        'hdmi' : 'vcap_hdmi'
    }
    devices = glob.glob('/dev/media*')
    for dev in devices:
        proc = subprocess.run(['media-ctl', '-d', dev, '-p'], capture_output=True, encoding='utf8')
        for line in proc.stdout.splitlines():
            if sources[src] in line:
                return dev

The ``parse_media_dv_detect`` function parse the digital video (DV) timing detected on the incoming HDMI stream. It returns the width, height, and framerate of the detected stream.

In [ ]:
def parse_media_dv_detect(dev):
    proc = subprocess.run(['media-ctl', '-d', dev, '-p'], capture_output=True, encoding='utf8')
    for line in proc.stdout.splitlines():
        if "dv.detect" in line:
            m = re.match(r'.*\s+(?P<w>\d+)x(?P<h>\d+)p(?P<f>\d+)\s+.*', line)
            return m.group('w'), m.group('h'), m.group('f')

Create a string representation of the pipeline by concatenating the individual element strings. The following sources are supported in this notebook:
* ``file`` : video file playback
* ``hdmi`` : HDMI input (platform3 only)

If ``hdmi`` is selected as source, ``mediasrcbin`` is instantiated for the video input and ``alsasrc`` is instantiated for the audio input from HDMI Rx. The video input is sent to ``kmssink`` and the audio input is sent to ``alsasink``. Each path has a capsfilter: the video caps are determined by parsing the DV detect info of the incoming HDMI stream; the audio caps are hard-coded. **Note:** This option is only valid for platform3.

If ``file`` is selected (default), the ``multifilesrc`` element in combination with ``decodebin`` is used to play back an encoded video file. Decodebin creates two src pads, one for video and one for audio which are dynamically connected to ``kmssink`` and ``alsasink`` respectively. The ``videoconvert`` and ``audioconvert`` elements are used to convert video and audio formats per the sinks' capabilities.

In [ ]:
source = "file" # change source to hdmi or file

if source == "hdmi":
    # get media device node
    media_device = get_media_dev_by_name(source)
    if media_device is None:
        raise Exception('Unable to find video source ' + source + '. Make sure the device is plugged in, powered, and the correct platform is used.')
    # parse dv detect string
    width, height, fps = parse_media_dv_detect(media_device)
    # create pipeline string
    vsrc = "mediasrcbin media-device=" + media_device
    vcaps = "video/x-raw, width=" + str(width) + ", height=" + str(height) + ", format=YUY2, framerate=" + fps + "/1"  
    asrc = "alsasrc device=hw:0,1"
    acaps = "audio/x-raw, rate=48000, channels=2, format=S16LE"
    pipe = vsrc + " ! " + vcaps + " ! queue max-size-bytes=0 ! " + vsink + " " + asrc + " ! " + acaps + " ! queue ! " + asink
else:
    # create pipeline string
    file_name = "/usr/share/movies/Big_Buck_Bunny_4K.webm.360p.vp9.webm"
    src = "multifilesrc loop=true location=" + file_name
    pipe = src + " ! decodebin name=dec dec. ! videoconvert ! queue max-size-bytes=0 ! " + vsink + " dec. ! audioconvert ! queue ! " + asink

print(pipe)

# 4. Create and Run the GStreamer Pipelines

Parse the string representations of the first and second pipeline as a single pipeline graph.

In [ ]:
pipeline = Gst.parse_launch(pipe)

The ``bus_call`` function listens on the bus for ``EOS`` and ``ERROR`` events. If any of these events occur, stop the pipeline (set to ``NULL`` state) and quit the main loop.

In case of an ``ERROR`` event, parse and print the error message.

In [ ]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    return True

Start the pipeline (set to ``PLAYING`` state), create the main loop and listen to messages on the bus. Register the ``bus_call`` callback function with the ``message`` signal of the bus. Start the main loop.

The video will be displayed on the monitor. 

To stop the pipeline, click the square shaped icon labelled 'Interrupt the kernel' in the top menu bar. Create a dot graph of the pipeline topology before stopping the pipeline. Quit the main loop.

In [ ]:
pipeline.set_state(Gst.State.PLAYING);

loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

try:
    loop.run()
except:
    sys.stdout.write("Interrupt caught\n")
    Gst.debug_bin_to_dot_file(pipeline, Gst.DebugGraphDetails.ALL, nb)
    pipeline.set_state(Gst.State.NULL)
    loop.quit()
    pass

# 5. View the Pipeline dot Graph

Register dot plugins for png export to work.

In [ ]:
!dot -c

Convert the dot file to png and display the pipeline graph. The image will be displayed below the following code cell. Double click on the generate image file to zoom in.

**Note:** This step may take a few seconds. Also, compared to previous notebooks, two disjoint graphs are displayed in the same image as we have created two parallel pipelines in this example.

In [ ]:
dotfile = dotdir + "/" + nb + ".dot"
graph = pydot.graph_from_dot_file(dotfile, 'utf-8')
display(Image(graph[0].create(None, 'png', 'utf-8')))

# 6. Summary

In this notebook you learned how to:
1. Create a GStreamer pipeline that includes video and audio from a string representation using the ``parse_launch()`` API
2. Export the pipeline topology as a dot file image and display it in the notebook

<center>Copyright© 2019 Xilinx</center>